In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `new-token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `new-toke

In [ ]:
!pip install streamlit transformers accelerate pyngrok

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_TOKEN")

In [ ]:
new_code = '''
import streamlit as st
from transformers import pipeline
from PIL import Image
import torch

@st.cache_resource
def load_pipe():
    device = 0 if torch.cuda.is_available() else -1
    return pipeline(
        "image-text-to-text",
        model="google/medgemma-4b-it",
        torch_dtype=torch.bfloat16,
        device=device
    )

pipe = load_pipe()

st.set_page_config(
    page_title="MedGemma Medical Analysis",
    page_icon=":hospital:",
    layout="centered"
)
st.title("MedGemma POC")
st.write("Upload a medical image and ask questions about it. This tool uses Google's MedGemma model for analysis.")

with st.sidebar:
    st.header("Information")
    st.write("""
    *Supported formats:* PNG, JPG, JPEG
    *Disclaimer:* This is for educational purposes only.
    """)
    st.header("Settings")
    max_image_size = st.slider("Max image size (pixels)", 512, 2048, 1024)

col1, col2 = st.columns([1, 1])
with col1:
    uploaded_file = st.file_uploader(
        "Upload medical image",
        type=["png", "jpg", "jpeg"]
    )
with col2:
    question = st.text_area(
        "Your question",
        value="Describe this medical image in detail",
        height=100
    )

if uploaded_file and question:
    try:
        image = Image.open(uploaded_file).convert("RGB")
        if max(image.size) > max_image_size:
            image.thumbnail((max_image_size, max_image_size), Image.Resampling.LANCZOS)
        st.image(image, caption="Uploaded Medical Image", use_container_width=True)
        st.caption(f"Image size: {image.size[0]}x{image.size[1]} pixels")
        if st.button("Analyze Image", type="primary"):
            with st.spinner("Analyzing medical image..."):
                messages = [
                    {"role": "system", "content": [{"type": "text", "text": "You are an expert radiologist."}]},
                    {"role": "user",   "content": [
                        {"type": "text",  "text": question},
                        {"type": "image", "image": image}
                    ]}
                ]
                output = pipe(text=messages, max_new_tokens=300)
                answer = output[0]["generated_text"][-1]["content"].strip()
            st.subheader("Analysis Results")
            st.markdown(f"""
            <div style="background-color: #F0F2F6; padding: 15px; border-radius: 10px; border-left: 4px solid #1F77B4;">
                {answer}
            </div>
            """, unsafe_allow_html=True)
            st.warning(":warning: *Medical Disclaimer:* This analysis is generated by an AI model and should not be used as a substitute for professional medical advice, diagnosis, or treatment.")
    except Exception as e:
        st.error(f"Error processing image: {str(e)}")

st.markdown("Powered by Google's MedGemma model via Hugging Face Transformers")
'''

In [ ]:
with open("app.py", "w") as f:
    f.write(new_code)

In [ ]:
!pkill -f streamlit
!pkill -f ngrok

In [ ]:
public_url = ngrok.connect(8501)
print(public_url)
!streamlit run app.py &

NgrokTunnel: "https://f9f8bbf61d76.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.184.85:8501

2025-07-17 06:24:51.158045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752733491.187610   27747 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752733491.200010   27747 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-17 06:24:51.253260: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio